In [42]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import sklearn.linear_model as sk
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import accuracy_score
%matplotlib inline
import matplotlib.pyplot as plt
import nltk
from nltk import tokenize
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import pickle
from wordcloud import WordCloud
from string import punctuation

In [43]:
#nltk.download("all") 

In [44]:
tweets=pd.read_csv("Datasets/imdb-reviews-pt-br.csv")

In [45]:
tweets.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


In [46]:
sentimento=tweets["sentiment"].replace(["neg","pos"],[0,1])

In [47]:
tweets["sentimento"]=sentimento
tweets.head()

,id,text_en,text_pt,sentiment,sentimento
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg,0
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg,0
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg,0
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg,0
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg,0


In [48]:
vetorizar = CountVectorizer(lowercase=False,max_features=50)
bag_of_words = vetorizar.fit_transform(tweets["text_pt"])

In [49]:
treino, teste, classe_treino, classe_teste = train_test_split(bag_of_words,
                                                              tweets["sentimento"],
                                                              random_state = 42)

In [50]:
todos_palavras = ' '.join([texto for texto in tweets["text_pt"]])

In [51]:
twweets_negativa = tweets.query("sentiment == 'neg'")
twweets_positivo = tweets.query("sentiment == 'pos'")

In [52]:
token_espaco = tokenize.WhitespaceTokenizer()
todas_palavras = ' '.join([texto for texto in tweets["text_pt"]])

In [53]:
palavras_irrelevantes = nltk.corpus.stopwords.words("portuguese")
frase_processada = list()

for opiniao in tweets["text_pt"]:
    nova_frase = list()
    palavras_texto = token_espaco.tokenize(opiniao)
    for palavra in palavras_texto:
        if palavra not in palavras_irrelevantes:
            nova_frase.append(palavra)
    frase_processada.append(' '.join(nova_frase))
    
tweets["tratamento_1"] = frase_processada

In [54]:
token_pontuacao = tokenize.WordPunctTokenizer()

In [55]:
pontuacao = list()
for ponto in punctuation:
    pontuacao.append(ponto)

pontuacao_stopwords = pontuacao + palavras_irrelevantes

frase_processada = list()
for opiniao in tweets["tratamento_1"]:
    nova_frase = list()
    palavras_texto = token_pontuacao.tokenize(opiniao)
    for palavra in palavras_texto:
        if palavra not in pontuacao_stopwords:
            nova_frase.append(palavra)
    frase_processada.append(' '.join(nova_frase))
    
tweets["tratamento_2"] = frase_processada

In [56]:
import unidecode
sem_acentos=[unidecode.unidecode(texto) for texto in tweets["tratamento_2"]]

In [57]:
tweets["tratamento_3"]=sem_acentos
frase_processada = list()
for opiniao in tweets["tratamento_3"]:
    nova_frase = list()
    palavras_texto = token_pontuacao.tokenize(opiniao)
    for palavra in palavras_texto:
        if palavra not in pontuacao_stopwords:
            nova_frase.append(palavra)
    frase_processada.append(' '.join(nova_frase))
    
tweets["tratamento_3"]=frase_processada

In [58]:
stopwords_sem_acento=[unidecode.unidecode(texto) for texto in pontuacao_stopwords]

In [59]:
frase_processada = list()
for opiniao in tweets["tratamento_3"]:
    nova_frase = list()
    opiniao=opiniao.lower()
    palavras_texto = token_pontuacao.tokenize(opiniao)
    for palavra in palavras_texto:
        if palavra not in stopwords_sem_acento:
            nova_frase.append(palavra)
    frase_processada.append(' '.join(nova_frase))
    
tweets["tratamento_4"]=frase_processada

In [60]:
stemmer=nltk.RSLPStemmer()

In [62]:
frase_processada = list()
for opiniao in tweets["tratamento_4"]:
    nova_frase = list()
    palavras_texto = token_pontuacao.tokenize(opiniao)
    for palavra in palavras_texto:
        nova_frase.append(stemmer.stem(palavra))
    frase_processada.append(' '.join(nova_frase))
    
tweets["tratamento_5"]=frase_processada

In [63]:
tfidf=TfidfVectorizer(lowercase=False)
vetor_tfidf=tfidf.fit_transform(tweets["tratamento_5"])

In [65]:
def classificar_texto(texto, coluna_texto, coluna_classificacao):
    vetorizar = CountVectorizer(lowercase=False, max_features=50)
    bag_of_words = vetorizar.fit_transform(texto[coluna_texto])
    treino, teste, classe_treino, classe_teste = train_test_split(bag_of_words,
                                                                 texto[coluna_classificacao],
                                                                 random_state = 42)
    
    regressao_logistica = LogisticRegression(solver="lbfgs")
    regressao_logistica.fit(treino, classe_treino)
    
    return regressao_logistica.score(teste, classe_teste)

In [66]:
#Sem Tratamento
acuracia_teste = classificar_texto(tweets, "text_pt", "sentimento")
print(acuracia_teste)

0.6583097452486858


In [67]:
#tratamento de Tirar palavras irelevantes
acuracia_teste = classificar_texto(tweets, "tratamento_1", "sentimento")
print(acuracia_teste)

0.6811160533764659


In [68]:
#Tratamento de Tirar pontuacao
acuracia_teste = classificar_texto(tweets, "tratamento_2", "sentimento")
print(acuracia_teste)

0.6892033966841893


In [69]:
acuracia_teste = classificar_texto(tweets, "tratamento_3", "sentimento")
print(acuracia_teste)

0.6887990295188031


In [70]:
acuracia_teste = classificar_texto(tweets, "tratamento_4", "sentimento")
print(acuracia_teste)

0.6874241811564901


In [71]:
acuracia_teste = classificar_texto(tweets, "tratamento_5", "sentimento")
print(acuracia_teste)

0.6954306510311363


In [74]:
regressao_logistica = LogisticRegression(solver="lbfgs")

In [76]:
tfidf=TfidfVectorizer(lowercase=False)
vetor_tfidf=tfidf.fit_transform(tweets["tratamento_5"])
treino,classe_treino = train_test_split(vetor_tfidf, tweets["sentimento"], random_state = 42)

regressao_logistica.fit(treino, classe_treino)
acuracia_tfidf = regressao_logistica.score(teste, classe_teste)
print(acuracia_tfidf)

0.8847553578649414


In [77]:
tweets["tratamento_5"]

<12365x69023 sparse matrix of type '<class 'numpy.float64'>'
	with 1174163 stored elements in Compressed Sparse Row format>

In [79]:
x = tweets["tratamento_5"].values
y = tweets["sentimento"].values

In [80]:
vectorizer = TfidfVectorizer()
train_vectors = vectorizer.fit_transform(x)

In [ ]:
model = MultinomialNB()
model.fit(x,y)

In [85]:
model = MultinomialNB()
model.fit(teste, classe_teste)

MultinomialNB()

In [93]:
model = LogisticRegression()
model.fit(teste, classe_teste)

LogisticRegression()

In [88]:
def tratamento(text):
    frase_processada = list()
    for opiniao in text:
        nova_frase = list()
        palavras_texto = token_pontuacao.tokenize(opiniao)
        for palavra in palavras_texto:
            if palavra not in pontuacao_stopwords:
                nova_frase.append(palavra)
        frase_processada.append(' '.join(nova_frase))
    return frase_processada

In [89]:
def classifica_tweet(text):
    test_vectors = vectorizer.transform(tratamento(text))
    return regressao_logistica.predict(test_vectors)

In [ ]:
def classifica_tweet1(text):
    test_vectors = vectorizer.transform(tratamento(text))
    return model1.predict(test_vectors)

In [94]:
import pickle

pickle.dump(model, open("lrModel", "wb"))
pickle.dump(vectorizer, open("vectorizer", "wb"))

In [95]:
model = pickle.load(open("lrModel", "rb"))
model = pickle.load(open("vectorizer", "rb"))

In [90]:
text=['como eu amo esses caras']

In [91]:
prediction = classifica_tweet(text)

print(prediction)

[1]


In [ ]:
text=['como eu odeio esses caras']

In [ ]:
prediction = classifica_tweet(text)

print(prediction)

[0]
